In [1]:
import decimal

In [4]:
class change_prec:
    def __init__(self, prec):
        self.prec = prec
        self.original_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        decimal.getcontext().prec = self.original_prec
        return False
    

In [7]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [10]:
with change_prec(4):
    print(decimal.Decimal(1)/decimal.Decimal(3))
print(decimal.Decimal(1)/decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [14]:
with decimal.localcontext() as ctx:
    ctx.prec = 1
    print(decimal.Decimal(1)/decimal.Decimal(3))
print(decimal.Decimal(1)/decimal.Decimal(3))

0.3
0.3333333333333333333333333333


In [15]:
from time import perf_counter, sleep

In [18]:
class Timer:
    def __init__(self):
        self.elapsed = 0
        
    def __enter__(self):
        self.start = perf_counter()
        return self
        
    def __exit__(self, exc_typ, exc_val, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [20]:
with Timer() as timer:
    sleep(5)
print(timer.start)
print(timer.stop)
print(timer.elapsed)

5092.527305829
5097.530788166
5.003482336999696


In [22]:
import sys

In [23]:
sys.stdout

In [33]:
class OutToFile:
    def __init__(self, fname):
        self.current_stdout = sys.stdout
        self.fname = fname
        
    def __enter__(self):
        self.file = open(self.fname, 'a')
        sys.stdout = self.file
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self.current_stdout
        return False

In [34]:
with OutToFile('test2.txt'):
    print('Testing....')

In [35]:
sys.stdout

In [36]:
with open('test2.txt', 'r') as f:
    print(f.readlines())

['Testing....\n']


In [49]:
class Tag:
    def __init__(self, tag):
        self.tag = tag
        
    def __enter__(self):
        print(f'<{self.tag}>', end ='')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f'</{self.tag}>', end='')
        return False

In [50]:
with Tag('p'):
    print('Few lines of', end = '')
    with Tag('b'):
        print('bold', end = '')
    print('text', end ='')

<p>Few lines of<b>bold</b>text</p>

In [55]:
class ListMaker:
    def __init__(self, title, indent=3, prefix='-'):
        self.title = title
        self.indent = indent
        self.prefix = prefix
        self.current_indent = 0
        print(self.title)
        
    def print(self, arg):
        print(f'{" " * self.current_indent}{self.prefix} {str(arg)}')
    
    def __enter__(self):
        self.current_indent += self.indent
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.current_indent -= self.indent
        return False

In [56]:
with ListMaker('Items') as lm:
    lm.print('Item 1')

Items
   - Item 1


In [58]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    with lm:
        lm.print('Item 1a')
        lm.print('Item 1b')
    lm.print('Item 2')
    with lm:
        lm.print('Item 2a')
        lm.print('Item 2b')
    

Items
   - Item 1
      - Item 1a
      - Item 1b
   - Item 2
      - Item 2a
      - Item 2b


In [60]:
with OutToFile('list.txt'):
    with ListMaker('Items') as lm:
        lm.print('Item 1')
        with lm:
            lm.print('Item 1a')
            lm.print('Item 1b')
        lm.print('Item 2')
        with lm:
            lm.print('Item 2a')
            lm.print('Item 2b')